### setup

In [1]:
import os
import argparse
import pickle
import time

import numpy as np; np.seterr(invalid='ignore')
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import TensorDataset, DataLoader

In [2]:
parser = {
    'data_path': '../data/wttsf/',
    'train_file': 'train_1.csv',
    'key_file': 'key_1.csv',
    'intermediate_path': '../intermediate/29082017/',
    'n_epoch': 3,
    'future': 70,
    'batch_size': 128,
    'hidden_size': 256,
    'log_every': 10,
    'read_from_file': True,
    'train': True,
    'model_file': '',  # if train = False
    'cuda': True,
    'seed': 29082017,
}
args = argparse.Namespace(**parser)

args.cuda = args.cuda and torch.cuda.is_available()
torch.manual_seed(args.seed);

### model

In [3]:
class DenseLSTMForecast(nn.Module):
    def __init__(self, hidden_size):
        super(DenseLSTMForecast, self).__init__()
        self.lstm1 = nn.LSTMCell(1, hidden_size, bias=False)
        self.lstm2 = nn.LSTMCell(hidden_size+1, hidden_size, bias=False)
#        self.lstm3 = nn.LSTMCell(2*hidden_size+1, hidden_size, bias=False)
        self.linear = nn.Linear(2*hidden_size+1, 1, bias=False)
        self.hidden_size = hidden_size

    def forward(self, x, future=1):
        o = []
        tt = torch.cuda if args.cuda else torch
        h1_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        c1_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        h2_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        c2_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
#        h3_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
#        c3_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        
        for x_t in x.chunk(x.size(1), dim=1):
            x_t = x_t.squeeze(dim=1)
            h1_t, c1_t = self.lstm1(x_t, (h1_t, c1_t))
            h1d_t = torch.cat([x_t, h1_t], dim=1)
            h2_t, c2_t = self.lstm2(h1d_t, (h2_t, c2_t))
            h2d_t = torch.cat([x_t, h1_t, h2_t], dim=1)
#            h3_t, c3_t = self.lstm3(h2d_t, (h3_t, c3_t))
#            h3d_t = torch.cat([x_t, h1_t, h2_t, h3_t], dim=1)
            o_t = self.linear(h2d_t)
            o.append(o_t)
            
        for i in range(future-1):
            h1_t, c1_t = self.lstm1(o_t, (h1_t, c1_t))
            h1d_t = torch.cat([o_t, h1_t], dim=1)
            h2_t, c2_t = self.lstm2(h1d_t, (h2_t, c2_t))
            h2d_t = torch.cat([o_t, h1_t, h2_t], dim=1)
#            h3_t, c3_t = self.lstm3(h2d_t, (h3_t, c3_t))
#            h3d_t = torch.cat([o_t, h1_t, h2_t, h3_t], dim=1)
            o_t = self.linear(h2d_t)
            o.append(o_t)

        return torch.stack(o, dim=1)

### utils

In [4]:
def smape(y_pred, y_true):
    raw_smape = np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred))
    kaggle_smape = np.nan_to_num(raw_smape)
    return np.mean(kaggle_smape) * 200

In [5]:
def get_data():
    raw_data_file = os.path.join(args.intermediate_path,
                                 'raw_data.pkl')
    scaled_data_file = os.path.join(args.intermediate_path,
                                    'scaled_data.pkl')
    scaler_file = os.path.join(args.intermediate_path, 'scaler.pkl')
    if not args.read_from_file:
        data_df = pd.read_csv(os.path.join(args.data_path, args.train_file),
                              index_col='Page')
        raw_data = np.nan_to_num(data_df.values.astype('float32'))
        data = np.log1p(raw_data)
        scaler = StandardScaler()
        scaler.fit(np.swapaxes(data[:, :-args.future], 0, 1))
        scaled_data = scaler.transform(np.swapaxes(data, 0, 1))
        scaled_data = np.swapaxes(scaled_data, 0, 1)
        
        with open(raw_data_file, 'wb') as f:
            pickle.dump(raw_data, f)
        with open(scaled_data_file, 'wb') as f:
            pickle.dump(scaled_data, f)
        with open(scaler_file, 'wb') as f:
            pickle.dump(scaler, f)
    else:
        with open(raw_data_file, 'rb') as f:
            raw_data = pickle.load(f)
        with open(scaled_data_file, 'rb') as f:
            scaled_data = pickle.load(f)
        with open(scaler_file, 'rb') as f:
            scaler = pickle.load(f)
    return raw_data, scaled_data, scaler

In [6]:
def train(raw_data, scaled_data, scaler, model, criterion, optimizer, epoch):
    print("=> EPOCH {}".format(epoch))
    p = np.random.permutation(raw_data.shape[0])
    inverse_p = np.argsort(p)
    
    input_tensor = torch.from_numpy(scaled_data[p, :-1]).unsqueeze(2)
    target_tensor = torch.from_numpy(scaled_data[p, 1:]).unsqueeze(2)
    dataset = TensorDataset(input_tensor, target_tensor)
    data_loader = DataLoader(dataset, args.batch_size)
    
    train_loss = 0
    val_output_list = []
    init_time = time.time()
    for i, (inputt, target) in enumerate(data_loader):
        if args.cuda:
            inputt = inputt.cuda()
            target = target.cuda()
        inputt = Variable(inputt)
        target = Variable(target)
        
        output = model(inputt)
        loss = criterion(output[:, :-args.future], target[:, :-args.future])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.data[0] * inputt.size(0)
        val_output_list.append(output[:,-args.future:]
                               .data.squeeze(2).cpu().numpy())
        
        if i % args.log_every == 0:
            print("   % Time: {:4.0f}s | Batch: {:3} | Loss: {:.4f}"
                  .format(time.time()-init_time, i+1, loss.data[0]))
        
    val_output_all = np.concatenate(val_output_list, axis=0)[inverse_p]
    prediction = np.swapaxes(scaler.inverse_transform(
            np.swapaxes(val_output_all, 0, 1)), 0, 1)
    prediction = np.exp(prediction) - 1
    prediction[prediction < 0.5] = 0
    
    train_loss /= raw_data.shape[0]
    val_loss = smape(prediction, raw_data[:, -args.future:])
    
    print("="*10)
    print("   % Epoch: {} | Time: {:4.0f}s | "
          "Train loss: {:.4f} | Val loss: {:.4f}"
          .format(epoch, time.time()-init_time, train_loss , val_loss))
    print("="*10)
    return val_loss

In [7]:
def forecast(scaled_data, scaler, model):
    input_tensor = torch.from_numpy(scaled_data).unsqueeze(2)
    target_tensor = torch.zeros(input_tensor.size(0))
    dataset = torch.utils.data.TensorDataset(input_tensor, target_tensor)
    data_loader = DataLoader(dataset, args.batch_size)
    
    output_list = []
    for inputt, _ in data_loader:
        if args.cuda:
            inputt = inputt.cuda()
        inputt = Variable(inputt)
        output = model(inputt, args.future)
        output_list.append(output.data.squeeze(2).cpu().numpy()
                           [:, -args.future:])
        
    output_all = np.concatenate(output_list, axis=0)
    prediction = np.swapaxes(scaler.inverse_transform(
            np.swapaxes(output_all, 0, 1)), 0, 1)
    prediction = np.exp(prediction) - 1
    prediction[prediction < 0.5] = 0
    return prediction

In [8]:
def save_model(model, epoch, val_loss):
    model_file = os.path.join(args.intermediate_path,
                              "model_seed{}_epoch{}_loss_{:.4f}.pth"
                              .format(args.seed, epoch, val_loss))
    torch.save(model.state_dict(), os.path.join(model_file))

### prepare

In [9]:
raw_data, scaled_data, scaler = get_data()

In [10]:
model = DenseLSTMForecast(args.hidden_size)
if args.cuda:
    model.cuda()
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters())
scheduler = MultiStepLR(optimizer, milestones=[2, 4])

### train

In [11]:
if args.train:
    for epoch in range(1, args.n_epoch+1):
        scheduler.step()
        val_loss = train(raw_data, scaled_data, scaler,
                         model, criterion, optimizer, epoch)
        save_model(model, epoch, val_loss)
else:
    model_file = os.path.join(args.intermediate_path, args.model_name)
    model.load_state_dict(torch.load(model_file))

=> EPOCH 1
   % Time:    3s | Batch:   1 | Loss: 0.7322
   % Time:   25s | Batch:  11 | Loss: 0.4500
   % Time:   46s | Batch:  21 | Loss: 0.5013
   % Time:   68s | Batch:  31 | Loss: 0.4756
   % Time:   89s | Batch:  41 | Loss: 0.4445
   % Time:  111s | Batch:  51 | Loss: 0.4530
   % Time:  133s | Batch:  61 | Loss: 0.4465
   % Time:  155s | Batch:  71 | Loss: 0.4276
   % Time:  177s | Batch:  81 | Loss: 0.4234
   % Time:  198s | Batch:  91 | Loss: 0.4439
   % Time:  220s | Batch: 101 | Loss: 0.4185
   % Time:  242s | Batch: 111 | Loss: 0.4310
   % Time:  264s | Batch: 121 | Loss: 0.4576
   % Time:  286s | Batch: 131 | Loss: 0.4137
   % Time:  308s | Batch: 141 | Loss: 0.4217
   % Time:  329s | Batch: 151 | Loss: 0.4203
   % Time:  351s | Batch: 161 | Loss: 0.3995
   % Time:  373s | Batch: 171 | Loss: 0.4767
   % Time:  394s | Batch: 181 | Loss: 0.4006
   % Time:  416s | Batch: 191 | Loss: 0.3908
   % Time:  438s | Batch: 201 | Loss: 0.4252
   % Time:  459s | Batch: 211 | Loss: 0.3989

### test

In [12]:
prediction = forecast(scaled_data, scaler, model)
prediction_file = os.path.join(args.intermediate_path,
                               'prediction_seed{}.pkl'.format(args.seed))

with open(prediction_file, 'wb') as f:
    pickle.dump(prediction, f)